<a href="https://colab.research.google.com/github/DIWEERAPURA/ML-Real-World-Practicals/blob/main/ZenML_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# Install ZenML and necessary dependencies
!pip install "zenml[server]"
!zenml integration install sklearn -y
!pip install pyparsing==2.4.2  # Required for Colab compatibility

import os
import IPython

# Restart the kernel automatically (recommended for Colab after installing packages)
IPython.Application.instance().kernel.do_shutdown(restart=True)


⠧ Installing integrations...


{'status': 'ok', 'restart': True}

In [1]:
NGROK_TOKEN =  "your token here"

In [2]:
from zenml.environment import Environment

if Environment.in_google_colab():  # Colab-only setup
    # Install and authenticate ngrok
    !pip install pyngrok
    !ngrok authtoken (NGROK_TOKEN)



/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `ngrok authtoken (NGROK_TOKEN)'


In [3]:
!rm -rf .zen
!zenml init



⠋ Initializing ZenML repository at /content.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.
ZenML repository initialized at /content.
⠋ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.

The local active stack was initialized to 'default'. This local configuration will only take effect 
when you're running ZenML from the initialized repository root, or from a subdirectory. For more 
information on repositories and configurations, please visit 
https://docs.zenml.io/user-guide/production-guide/understand-stacks.


In [4]:
# Scikit-Learn SVC classifier to clasiffy Images of handwritten digits, We first load the data, trauin a model on the training data set and then test it on the test set.
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split



def train_test() -> None:
#Train and test a Scikit-learn SVC classifier on digits
  digits = load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(
  data, digits.target, test_size=0.2, shuffle=False)

  model = SVC(gamma=0.001)
  model.fit(X_train, y_train)
  test_acc = model.score(X_test, y_test)
  print(f"Test accuracy: {test_acc}")

train_test()

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


NumExpr defaulting to 2 threads.
Test accuracy: 0.9583333333333334


In [5]:
#turning experiments to pipelines with ZenML
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer() -> Tuple[
  Annotated[np.ndarray, "X_train" ],
  Annotated[np.ndarray, "X_test"],
  Annotated[np.ndarray, "y_train"],
  Annotated[np.ndarray, "y_test"],
  ]:
  #Load the digits dataset as numpy arrays.
  digits = load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(
  data, digits.target, test_size=0.2, shuffle=False)

  return X_train, X_test, y_train, y_test

@step
def svc_trainer(
  X_train: np.ndarray,
  y_train: np.ndarray,
) -> ClassifierMixin:
#Train an sklearn SVC classifier.
  model = SVC(gamma=0.001)
  model.fit(X_train, y_train)
  return model
@step
def evaluator(
  X_test: np.ndarray,
  y_test: np.ndarray,
  model: ClassifierMixin,
)->float:
#Calculate the test set accuracy of an sklearn model.
  test_acc = model.score(X_test, y_test)
  print(f"Test accuracy: {test_acc}")
  return test_acc

In [6]:
from zenml import pipeline

@pipeline
def digits_pipeline():
#Links all the steps together in a pipeline"""
  X_train, X_test, y_train, y_test = importer( )
  model = svc_trainer (X_train=X_train, y_train=y_train)
  evaluator (X_test=X_test, y_test=y_test, model=model)

In [7]:
digits_svc_pipeline = digits_pipeline()
# digits_svc_pipeline.run(unlisted=True)



Initiating a new run for the pipeline: digits_pipeline.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
Dashboard URL for Pipeline Run: http://127.0.0.1:8237/runs/efd57306-fe26-4f16-83db-58a4cc644a59
Using cached version of step importer.
Using cached version of step svc_trainer.
Using cached version of step evaluator.
All steps of the pipeline run were cached.


In [8]:
# Install dependencies
!pip install zenml pyngrok

# Import necessary libraries
from pyngrok import ngrok

# Set your Ngrok authentication token
NGROK_AUTH_TOKEN = "your token here"  # Replace with your actual token
!ngrok authtoken $NGROK_AUTH_TOKEN

# Start Ngrok tunnel on port 8237
port = 8237  # Adjust if needed
public_url = ngrok.connect(port).public_url
print(f"Ngrok Tunnel URL: {public_url}")

# Start ZenML server
!zenml up --port=8237


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


INFO:pyngrok.ngrok:Opening tunnel named: http-8237-9c52ae88-b4eb-40b6-9a10-627a5f4b193b


Opening tunnel named: http-8237-9c52ae88-b4eb-40b6-9a10-627a5f4b193b


INFO:pyngrok.process.ngrok:t=2025-03-15T14:54:35+0000 lvl=info msg="no configuration paths supplied"


t=2025-03-15T14:54:35+0000 lvl=info msg="no configuration paths supplied"


...

t=2025-03-15T14:54:35+0000 lvl=info msg=start pg=/api/tunnels id=e11f293e5ca98f14


INFO:pyngrok.process.ngrok:t=2025-03-15T14:54:35+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-9c52ae88-b4eb-40b6-9a10-627a5f4b193b addr=http://localhost:8237 url=https://ff27-34-145-21-243.ngrok-free.app


Ngrok Tunnel URL: https://ff27-34-145-21-243.ngrok-free.app
t=2025-03-15T14:54:35+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-9c52ae88-b4eb-40b6-9a10-627a5f4b193b addr=http://localhost:8237 url=https://ff27-34-145-21-243.ngrok-free.app


INFO:pyngrok.process.ngrok:t=2025-03-15T14:54:35+0000 lvl=info msg=end pg=/api/tunnels id=e11f293e5ca98f14 status=201 dur=67.800464ms


t=2025-03-15T14:54:35+0000 lvl=info msg=end pg=/api/tunnels id=e11f293e5ca98f14 status=201 dur=67.800464ms
The `zenml up` command is deprecated and will be removed in a future release. Please use the `zenml 
login --local` command instead.
Calling `zenml login --local`...
The local daemon ZenML server is already configured with the same parameters.
⠋ Starting service 'DaemonZenServer[02d4a002-b4dc-41fd-acb1-d5532176b1b0] (type: zen_server, flavor:
daemon)'.

Connecting to the local daemon ZenML server (http://127.0.0.1:8237).
Updated the global store configuration.
Connected to the local daemon ZenML server (http://127.0.0.1:8237).
The local ZenML dashboard is available at 'http://127.0.0.1:8237'.


In [9]:

# Install dependencies
!pip install zenml pyngrok

# Import necessary libraries
from pyngrok import ngrok

# Set your Ngrok authentication token
NGROK_AUTH_TOKEN = "your token here"  # Replace with your actual token
!ngrok authtoken $NGROK_AUTH_TOKEN

# Start Ngrok tunnel on port 8237
port = 8237  # Adjust if needed
public_url = ngrok.connect(port).public_url
print(f"Ngrok Tunnel URL: {public_url}")

# Use ZenML to login in local mode
!zenml login --local


INFO:pyngrok.process.ngrok:t=2025-03-15T14:54:53+0000 lvl=info msg="join connections" obj=join id=a064fc051f9f l=127.0.0.1:8237 r=175.157.18.169:17753


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
t=2025-03-15T14:54:53+0000 lvl=info msg="join connections" obj=join id=a064fc051f9f l=127.0.0.1:8237 r=175.157.18.169:17753


INFO:pyngrok.ngrok:Opening tunnel named: http-8237-ec699d13-898b-4dae-addc-55f50b47df23


Opening tunnel named: http-8237-ec699d13-898b-4dae-addc-55f50b47df23


INFO:pyngrok.process.ngrok:t=2025-03-15T14:54:53+0000 lvl=info msg=start pg=/api/tunnels id=9ccf9bd50c8e453b


...

t=2025-03-15T14:55:09+0000 lvl=info msg="join connections" obj=join id=17000ac930aa l=127.0.0.1:8237 r=175.157.18.169:17753


INFO:pyngrok.process.ngrok:t=2025-03-15T14:55:09+0000 lvl=info msg="join connections" obj=join id=a076a7b823a1 l=127.0.0.1:8237 r=175.157.18.169:17753


t=2025-03-15T14:55:09+0000 lvl=info msg="join connections" obj=join id=a076a7b823a1 l=127.0.0.1:8237 r=175.157.18.169:17753


INFO:pyngrok.process.ngrok:t=2025-03-15T14:55:09+0000 lvl=info msg="join connections" obj=join id=e46cfe59b034 l=127.0.0.1:8237 r=175.157.18.169:17753


t=2025-03-15T14:55:09+0000 lvl=info msg="join connections" obj=join id=e46cfe59b034 l=127.0.0.1:8237 r=175.157.18.169:17753
The local daemon ZenML server is already configured with the same parameters.
⠋ Starting service 'DaemonZenServer[02d4a002-b4dc-41fd-acb1-d5532176b1b0] (type: zen_server, flavor:
daemon)'.

Connecting to the local daemon ZenML server (http://127.0.0.1:8237).
Updated the global store configuration.
Connected to the local daemon ZenML server (http://127.0.0.1:8237).
The local ZenML dashboard is available at 'http://127.0.0.1:8237'.
